# Parsing Rain Data Files

The purpose of the following code is to parse the Rain data from the <i>The Alberta Hail Project Meteorological and Barge-Humphries Radar Archive</i> and output it to a `.CSV` file.

The data consists of rain measurements and observations submitted by Alberta farmers between the period of 1974 and 1985. 

The data exists as a directory of `.DAT` files which were manually converted to `.TXT` files (by changing the extension). Then, using regular expressions (also known as regex), the data is split apart and recombined into a nested list format allowing for export to a `.CSV` file.

-----

This code imports the modules and defines the custom functions that are needed to parse the rain data. The comments coloured in light blue and red describe the individual code.

In [1]:
import re
import csv
from os import listdir
from os.path import splitext
from os.path import basename

# This function works on the contents of the files

def read_file(filename):
    "Read the contents of FILENAME and return as a string."
    infile = open(filename)
    contents = infile.read()
    infile.close()
    return contents

# These functions remove the the path and file extension from a filename

def list_textfiles(directory):
    "Return a list of filenames ending in '.txt'"
    textfiles = []
    for filename in listdir(directory):
        if filename.endswith(".txt"):
            textfiles.append(directory + "/" + filename)
    return textfiles


def remove_ext(filename):
    "Removes the file extension, such as .txt"
    name, extension = splitext(filename)
    return name


def remove_dir(filepath):
    "Removes the path from the file name"
    name = basename(filepath)
    return name


def get_filename(filepath):
    "Removes the path and file extension from the file name"
    filename = remove_ext(filepath)
    name = remove_dir(filename)
    return name

Below is a list of the files in the Rain directory that need to be parsed. This list is helpful because we will need to manually change the filename in the next section of code in order to parse each file separately. 

The function `listdir` is called to display the files contained in the specified directory, shown below in red text.

In [2]:
print(listdir('Rain/'))

['74RAIN.txt', '75RAIN.txt', '76RAIN.txt', '77RAIN.txt', '78RAIN.txt', '79RAIN.txt', '80RAIN.txt', '81RAIN.txt', '82RAIN.txt', '83RAIN.txt', '84RAIN.txt', '85RAIN.txt']


This is where the raw data is imported and assigned to a variable. To read a different file from the directory, the filename needs to be changed (based on the above list). It's important to note that the original data is never altered, only the data that is loaded and assigned to the `file` variable.

In [3]:
file = 'Rain/76RAIN.txt'

This code calls on a custom function `get_filename` to strip the path and file extension from the filename, storing it in the variable `name`. This variable will be used later to name the `.CSV` file so it matches the name of the `.TXT` file from which the data came. 

Then, the data from the variable `file` is read using the custom function `read_file` and passed to a regex that matches the format of the file (seen in red text).

In [4]:
name = get_filename(file)
text = read_file(file)

data = re.findall(r'(\s)(\d{1})(\d{2})(\d{1})(\d{2})(\d{4})(\d{2})(\d{2})(\d{2})(\d{1})(\d{4})(\d{3})(\d{3})', text, re.MULTILINE)

Here's a look at the original data format. Each string of numbers is a separate entry consisting of groups of numbers that correspond to a specific data point, ranging from dates, times, geographical information, and rain duration.

In [5]:
print(file)
print(text[:113])

Rain/76RAIN.txt
 07663011010901144
 07662511013026025
 07662411012440015
 07660101110108174
 07660200011362015
 07660201112362015


Here is the data formatted by the regex. The first digit indicates the origin of the report (via mail-in card, etc.), the second is the year, and the third and fourth are the month and day. The data was parsed according to the information provided by the accompanying <a href= 'hailcode.html' target=_blank>codebook</a>.

In [6]:
print(data[:1])

[(' ', '0', '76', '6', '05', '0010', '02', '42', '22', '4', '1830', '999', '184')]


Now the data can be converted to `.CSV`. This code uses the `name` variable that was created earlier to name the file and writes each group of data to one line in the file. 

In [7]:
with open(name + '.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(data)

-----

This code reads all of the files and prints each line to one master `.CSV` file. The only extra step required is in the fourth chunk of code, where the master list needs to be flattened so that each string of data appears in its own row in the file.

In [8]:
filenames = []
for files in list_textfiles('Rain/'):
    files = get_filename(files)
    filenames.append(files)

In [9]:
docs = []
for filename in list_textfiles('Rain/'):
    docs.append(read_file(filename))

In [10]:
data_2 = []
for doc in docs:
    data_2.append(re.findall(r'(\s)(\d{1})(\d{2})(\d{1})(\d{2})(\d{4})(\d{2})(\d{2})(\d{2})(\d{1})(\d{4})(\d{3})(\d{3})', doc, re.MULTILINE))

In [11]:
alldata = [line for sublist in data_2 for line in sublist]

In [12]:
with open('allRain.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(alldata)